# Building a Classifier App with weblink
> Creating an image classifier app that is on the net! Using Jupyter notebooks, FastAi, Binder and Voila

- toc:true
- branch: master
- badges: true
- comments: true
- author: Thomas H. Simm
- categories: [fastai, jupyter, Binder, Voila, myBinder, ThomasHSimm]

1) The Model Part

2) The Python Part Of the App

3) The Binder Part

## 1) The model Part

The model was run on https://colab.research.google.com with a GPU

Necessary to have a GPU for time

Some imports and installs

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
from fastbook import *
from fastai.vision.widgets import *

Gets Azure search key to use Bing search API

In [ ]:

key = os.environ.get('AZURE_SEARCH_KEY', 'keygoeshere')

I'm going to do a classifier for holiday types

In [ ]:
destas={'beach':{'beach','tropical','sea','beach holidays'},
        'snow':{'ski','snowboard','snow','ski holidays'},
        'countryside':{'lakes mountains','countryside','forest','fields'},
        'city':{'city','cities','bars','buildings'},
        'safari':{'safari','safari holidays','safari park','safari africa'}}

Creates a folder containing images for each type

Need to add to dir with different searches not delete and add new stuff each time

For eaach holiday type- go through the search topics and add the results together before downloading images to the holiday type folder

This gives 600 pics per holiday type

In [ ]:
path = Path('Destinations')
try:
    path.mkdir()
except:
    pass

In [ ]:

# resultsALL
for o in destas:
    dest = (path/o)
    dest.mkdir(exist_ok=True)
    for ii,oo in enumerate(destas[o]): 
        # print(oo)
        results = search_images_bing(key, f'{oo}')
        if ii>0:
            resultsALL = L(resultsALL,results).concat()
        else:
            resultsALL=results
    print(o)
    print(np.shape(resultsALL))
    download_images(dest, urls=resultsALL.attrgot('contentUrl'))


Create a data block for fastai

In [ ]:
fns = get_image_files(path)

dests = DataBlock(
    blocks=(ImageBlock, CategoryBlock), 
    get_items=get_image_files, 
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    get_y=parent_label,
    item_tfms=Resize(128))

get rid of failed images

In [ ]:
failed = verify_images(fns)
failed.map(Path.unlink)
failed

Have a look at the images


In [ ]:
dls = dests.dataloaders(path)
dls.valid.show_batch(max_n=40, nrows=10)

![]({{ site.baseurl }}/images/ic1.png)

create a dls for the learner

- randomresizedcrop
- default aug transforms


In [ ]:
dests = dests.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())
dls = dests.dataloaders(path)

In [ ]:
Do the learning over 4 epochs